## Demo Structured Output (JSON Mode)

- https://docs.litellm.ai/docs/completion/json_mode

In [1]:
import json
import os
from textwrap import dedent

from litellm import completion, get_supported_openai_params
from pydantic import BaseModel, Field
from rich.pretty import pprint

In [2]:
class TruckModel(BaseModel):
    number_of_axles: int = Field(
        ...,
        description="Number of axles in the truck.",
    )
    truck_height_in_meters: float = Field(
        ...,
        description="Height of the truck in meters.",
    )
    truck_width_in_meters: float = Field(
        ...,
        description="Width of the truck in meters.",
    )

In [3]:
# pprint(TruckModel.model_json_schema(), expand_all=True)

In [4]:
# for _ in TruckModel.model_fields.values():
#     pprint(_, expand_all=True)

In [5]:
params = get_supported_openai_params(
    model="phi4:14b-q8_0",
    custom_llm_provider="ollama_chat",
)

pprint(params, expand_all=True)

print(f"{'response_format' in params=}")

[
│   'max_tokens',
│   'max_completion_tokens',
│   'stream',
│   'top_p',
│   'temperature',
│   'seed',
│   'frequency_penalty',
│   'stop',
│   'tools',
│   'tool_choice',
│   'functions',
│   'response_format'
]

'response_format' in params=True


In [6]:
response = completion(
    model="ollama_chat/phi4:14b-q8_0",
    # model="azure/gpt-4o-mini",
    # api_base=os.environ["AZURE_API_URL"] + "/gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": dedent(
                """
You are an expert in extracting structured entities from a given >>>CONTENT<<<.

Instructions:
•	Output only the JSON document—do not include any markdown, extra spaces, or newline characters.
•	Follow the provided JSON schema precisely.
•	If an entity is not found, return 0 for integers and an empty string ("") for text fields.
•	Ensure all required fields are present in the output.                
            """
            ).strip(),
        },
        {
            "role": "user",
            "content": dedent(
                """
            Extract the entities from >>>for a vehicle that is 2.54 meters high, 6.2 feet wide and with 4 axles<<<
            """
            ).strip(),
        },
    ],
    response_format=TruckModel,
)

In [7]:
pprint(
    json.loads(response.choices[0].message.content),
    expand_all=True,
)

{
│   'number_of_axles': 4,
│   'truck_height_in_meters': 2.54,
│   'truck_width_in_meters': 1.89216
}

In [15]:
response = completion(
    model="azure/gpt-4o-mini",
    api_base=os.environ["AZURE_API_URL"] + "/gpt-4o-mini",
    temperature=0.0,
    messages=[
        {
            "role": "system",
            "content": dedent(
                """
Extract the truck entities from >>>CONTENT<<< and emit valid JSON document.
            """
            ).strip(),
        },
        {
            "role": "user",
            "content": dedent(
                """
            >>>Find route from San Diego to San Francisco
            for a truck that is 2.54 meters high, 6.2 feet wide and with 5 axles<<<
            """
            ).strip(),
        },
    ],
    response_format=TruckModel,
)

pprint(
    json.loads(response.choices[0].message.content),
    expand_all=True,
)

{
│   'number_of_axles': 5,
│   'truck_height_in_meters': 2.54,
│   'truck_width_in_meters': 1.89
}